In [ ]:
#default_exp builtins

In [28]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script()

Converted 00__init__.ipynb.
Converted 00_core.ipynb.
Converted 01_hdict.ipynb.
Converted 02_seqs.ipynb.
Converted 03_utils.ipynb.
Converted 04_builtins.ipynb.
Converted index.ipynb.


In [25]:
ls -lh dask_rnaseq/

total 80K
-rw-r--r-- 1 jupyter-ken jupyter-ken  17K Mar 31 21:40 builtins.py
-rw-r--r-- 1 jupyter-ken jupyter-ken  12K Mar 31 21:40 core.py
-rw-r--r-- 1 jupyter-ken jupyter-ken 5.3K Mar 31 21:40 hdict.py
-rw-r--r-- 1 jupyter-ken jupyter-ken  17K Mar 31 21:40 __init__.py
-rw-r--r-- 1 jupyter-ken jupyter-ken 2.1K Mar 31 21:40 _nbdev.py
drwxr-xr-x 2 jupyter-ken jupyter-ken 4.0K Mar 31 21:40 __pycache__/
-rw-r--r-- 1 jupyter-ken jupyter-ken 1.1K Mar 31 21:40 seqs.py
-rw-r--r-- 1 jupyter-ken jupyter-ken 6.8K Mar 31 21:40 utils.py


In [17]:
#export
import os
import glob

import gzip
import numpy as N
import pandas as PD

from prefect import task, Flow, Parameter, unmapped, case

from dask_rnaseq.core import FileInFileOut, HBox, ptoml
from dask_rnaseq.utils import *

In [3]:
%load_ext autoreload
%autoreload 2

# Act on FASTQ

## Countreads

In [2]:
# export
class CountReads(FileInFileOut):
    """
    Inputs: 
        1. Fastq (p.r1)
    Outputs:
        1. count.txt (=>p.info.nreads)
    
    """
    nickname = 'countreads'
    i_tmpls = dict(
        r1 = '{r1}'
    )
    o_tmpls = dict(
        o1 = '{r1}.count.txt'
    )
            
    def _run(self):
        cmd = f"cat {self._inps['r1']} | wc -l > {self._outs['o1']}"
        subprocesscall(cmd)
        
    def _read_stats(self):
        nlines = int(open(self.outs['o1']).read())
        return dict(nreads = int(nlines/4))

## Flexbar

In [3]:
# export
class Flexbar(FileInFileOut):
    """
    Inputs: 
        1. Fastq (p.r1, [p.r2])
    Outputs: 
        1. Fastq (=>p.r1, [p.r2])
    """
    nickname = 'flexbar'
    t_tmpls = dict(
        pr1 ='{prefix}_1.fastq', 
        pr2 ='{prefix}_2.fastq',
        r1  ='{prefix}.fastq'
    )
    o_tmpls = dict(
        pr1 ='{prefix}_1.fastq.gz', 
        pr2 ='{prefix}_2.fastq.gz', 
        r1  ='{prefix}.fastq.gz',
        log ='{prefix}.log'
    )
    
    def __init__(self, preset='Nextera', **kw):
        self.preset = preset        
        super().__init__(tag=preset, **kw)
                
    def _define_files(self):
        p = self.p
        self.paired = paired = (p.r2 != '')
        if paired:
            ikeys = ['r1','r2']
            tkeys = ['pr1','pr2']
            okeys = ['pr1','pr2','log']
        else:
            ikeys = ['r1']
            tkeys = ['r1']
            okeys = ['r1','log']
        self.inps = {k: self.i_tmpls[k].format(**p) for k in ikeys}
        self.tmps = {k: self.t_tmpls[k].format(**p) for k in tkeys}            
        self.outs = {k: self.o_tmpls[k].format(**p) for k in okeys}
                    
    def _read_stats(self):
        lines = open(self.outs['log']).readlines()
        dic = dict(
            processed = int([x for x in lines if x.startswith('Processed reads')][0].split()[2]),
            remaining = int([x for x in lines if x.startswith('Remaining reads')][0].split()[2]),
        )
        return dic    
    
    def _modify_param_specific(self):
        # swap inputs
        pa = self.param
        o = self.outs
        if self.paired:
            pa.r1, pa.r2 = o['pr1'], o['pr2']
        else:
            pa.r1 = o['r1']
            
    def _run(self):
        flexbar = self.pcfg.get('execpath','flexbar')
        nt = self.get_nthreads()
        i,o,t = self._inps, self._outs, self._tmps
        if self.paired:
            cmd = f"{flexbar} -r {i['r1']} -p {i['r2']} -aa {self.preset} -ap ON -n {nt} -t {self._prefix}"
        else:
            cmd = f"{flexbar} -r {i['r1']} -aa {self.preset} -n {nt} -t {self._prefix}" 
        subprocesscall(cmd) # have to redirect
        # compress with pigz
        self.compress(self._tmps)
                

## Flexbar tests

In [4]:
cfg = HBox.from_toml("""
nfs = './tests'
ramdisk = './tests2'
odir = './tests/outputs'
idir = './tests'
nfshost = 'host1'
sshuser = 'user1'
workers = ['host1']
force = true
did = 'datasetid'
maxthreads = 8

[analysis]
execpath = 'nonexistent'

[analysis.tag]
extra = 'param1'

[flexbar]
execpath = '/mnt/p1_nvme4/envs/bio/bin/flexbar'
[pigz]
execpath = '/mnt/p1_nvme4/envs/bio/bin/pigz'
""")

fb = Flexbar(preset='Nextera', cfg=cfg)

sample = dict(sid = 'S69_R1',
              r1 = './tests/5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz',
              r2 = './tests/5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz')
fb.set_param(sample)
ptoml(fb.p)

nfs = "./tests"
ramdisk = "./tests2"
odir = "./tests/outputs"
idir = "./tests"
nfshost = "host1"
sshuser = "user1"
workers = [ "host1",]
force = true
did = "datasetid"
maxthreads = 8
sid = "S69_R1"
r1 = "./tests/5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz"
r2 = "./tests/5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz"

[analysis]
execpath = "nonexistent"

[flexbar]
execpath = "/mnt/p1_nvme4/envs/bio/bin/flexbar"

[pigz]
execpath = "/mnt/p1_nvme4/envs/bio/bin/pigz"

[analysis.tag]
extra = "param1"



In [7]:
fb._prep_dstdir()
fb._define_files()
fb.inps, fb.outs, fb.tmps

({'r1': './tests/5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz',
  'r2': './tests/5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz'},
 {'pr1': './tests/outputs/datasetid/flexbar.Nextera/S69_R1_1.fastq.gz',
  'pr2': './tests/outputs/datasetid/flexbar.Nextera/S69_R1_2.fastq.gz',
  'log': './tests/outputs/datasetid/flexbar.Nextera/S69_R1.log'},
 {'pr1': './tests/outputs/datasetid/flexbar.Nextera/S69_R1_1.fastq',
  'pr2': './tests/outputs/datasetid/flexbar.Nextera/S69_R1_2.fastq'})

In [15]:
fb.run(sample)

********** NFS => RAMDISK *********************************
./tests/5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz=>suginok-p1:./tests2/5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz
***********************************************************
********** NFS => RAMDISK *********************************
./tests/5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz=>suginok-p1:./tests2/5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz
***********************************************************
SHELL CMD ----------------------------------
/mnt/p1_nvme4/envs/bio/bin/flexbar -r ./tests2/5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz -p ./tests2/5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz -aa Nextera -ap ON -n 8 -t ./tests2/outputs/datasetid/flexbar.Nextera/S69_R1
--------- ----------------------------------
SHELL CMD ----------------------------------
/mnt/p1_nvme4/envs/bio/bin/pigz -p 8 --fast ./tests2/outputs/datasetid/flexbar.Nextera/S69_R1_1.fastq
--------- ----------------------------------
SHELL CMD ----------------

<HBox: {'sid': 'S69_R1', 'r1': './tests/outputs/datasetid/flexbar.Nextera/S69_R1_1.fastq.gz', 'r2': './tests/outputs/datasetid/flexbar.Nextera/S69_R1_2.fastq.gz', 'delete': [('suginok-p1', './tests2/5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz'), ('suginok-p1', './tests2/5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz'), ('suginok-p1', './tests2/outputs/datasetid/flexbar.Nextera/S69_R1_1.fastq.gz'), ('suginok-p1', './tests2/outputs/datasetid/flexbar.Nextera/S69_R1_2.fastq.gz'), ('suginok-p1', './tests2/outputs/datasetid/flexbar.Nextera/S69_R1.log')], 'flexbar': {'Nextera': {'dstdir': './tests/outputs/datasetid/flexbar.Nextera', 'prefix': './tests/outputs/datasetid/flexbar.Nextera/S69_R1', 'inputs': ['5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz', '5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz'], 'outputs': ['S69_R1_1.fastq.gz', 'S69_R1_2.fastq.gz', 'S69_R1.log'], 'temps': ['S69_R1_1.fastq', 'S69_R1_2.fastq']}}, 'info': {'flexbar': {'Nextera': {'processed': 18384, 'remaining': 18368}}}, 'history': 

In [16]:
ptoml(fb.param['flexbar'])

[Nextera]
dstdir = "./tests/outputs/datasetid/flexbar.Nextera"
prefix = "./tests/outputs/datasetid/flexbar.Nextera/S69_R1"
inputs = [ "5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz", "5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz",]
outputs = [ "S69_R1_1.fastq.gz", "S69_R1_2.fastq.gz", "S69_R1.log",]
temps = [ "S69_R1_1.fastq", "S69_R1_2.fastq",]



In [17]:
cat ./tests2/outputs/datasetid/flexbar.Nextera/S69_R1.log


               ________          __              
              / ____/ /__  _  __/ /_  ____ ______
             / /_  / / _ \| |/ / __ \/ __ `/ ___/
            / __/ / /  __/>  </ /_/ / /_/ / /    
           /_/   /_/\___/_/|_/_.___/\__._/_/     

Flexbar - flexible barcode and adapter removal, version 3.5.0
Developed with SeqAn, the library for sequence analysis

Available on github.com/seqan/flexbar


Local time:            Tue Mar 30 17:10:23 2021

Number of threads:     8
Bundled fragments:     256

Target name:           ./tests2/outputs/datasetid/flexbar.Nextera/S69_R1
File type:             fastq
Reads file:            ./tests2/5d_GH146mtdT_PN_Lee_L1_S69_R1_001.fastq.gz
Reads file 2:          ./tests2/5d_GH146mtdT_PN_Lee_L1_S69_R2_001.fastq.gz   (paired run)
Adapter preset:        Nextera

max-uncalled:          0
min-read-length:       18

adapter-pair-overlap:  ON
adapter-trim-end:      RIGHT
adapter-min-overlap:   3
adapter-min-poverlap:  40
adapter-error-rate:    0.1
ada

# Mapper

## Dart

In [15]:
# export
class Dart(FileInFileOut):
    """
    Inputs: 
        1. fastq (p.r1, [p.r2])
    Outputs: 
        1. aligned.sorted.bam (=>p.bam)
        2. unmapped.fastq (=>p.r1, [p.r2])
    """
    nickname = 'dart'
    t_tmpls = dict(
        obam = '{prefix}.bam', 
        abam = '{prefix}.aligned.bam',
        ubam = '{prefix}.unmapped.bam',
        fq1 =  '{prefix}.{indexname}.unmapped_1.fastq',
        fq2 =  '{prefix}.{indexname}.unmapped_2.fastq',
        fq =   '{prefix}.{indexname}.unmapped.fastq',
    )
    o_tmpls = dict(
        sbam = '{prefix}.aligned.sorted.bam', 
        sbai = '{prefix}.aligned.sorted.bam.bai', 
        log =  '{prefix}.{indexname}.log',
        fq1 =  '{prefix}.{indexname}.unmapped_1.fastq.gz',
        fq2 =  '{prefix}.{indexname}.unmapped_2.fastq.gz',                   
        fq =   '{prefix}.{indexname}.unmapped.fastq.gz'
    )
    
    def __init__(self, indexname, **kw):
        self.indexname = indexname
        super().__init__(tag=indexname, **kw)
        self.cfg.indexname = indexname
        
    def _define_files(self):
        p = self.p
        self.paired = paired = (p.r2 != '')
        if paired:
            ikeys = ['r1','r2']
            tkeys = ['obam','abam','ubam','fq1','fq2']
            okeys = ['sbam','sbai','log','fq1','fq2']
        else:
            ikeys = ['r1']
            tkeys = ['obam','abam','ubam','fq']
            okeys = ['sbam','sbai','log','fq']
        self.inps = {k: self.i_tmpls[k].format(**p) for k in ikeys}
        self.tmps = {k: self.t_tmpls[k].format(**p) for k in tkeys}            
        self.outs = {k: self.o_tmpls[k].format(**p) for k in okeys}
    
    def _read_stats(self):
        log = self.outs['log']
        dic = {}
        def _extract_num(line):
            return [int(s) for s in line.split() if s.isdigit()]
        for line in open(log,'rt'):
            if line.startswith('All the '):
                dic['total_reads'],dic['duration'] = _extract_num(line)
            elif line.startswith('\t# of total mapped reads'):
                dic['mapped'] = _extract_num(line)[0]
            elif line.startswith('\t# of paired sequences'):
                dic['paired'] = _extract_num(line)[0]
            elif line.startswith('\t# of unique mapped'):
                dic['unique'] = _extract_num(line)[0]
            elif line.startswith('\t# of multiple mapped'):
                dic['multimap'] = _extract_num(line)[0]
            elif line.startswith('\t# of unmapped reads'):
                dic['unmapped'] = _extract_num(line)[0]
            elif line.startswith('\t# of splice junctions'):
                dic['spliced'] = _extract_num(line)[0]
        return dic 
    
    def _modify_param_specific(self):
        # swap fastq inputs
        pa = self.param
        o = self.outs
        if self.paired:
            pa.r1, pa.r2 = o['fq1'], o['fq2']
        else:
            pa.r1 = o['fq']
        # bam
        pa.bam = o['sbam'] # sorted bam
        
    def _run0(self):
        p = self.p
        pc = self.pcfg
        i = self._inps
        t = self._tmps
        o = self._outs
        paired = self.paired
        # 1st step: bam
        dart = self.pcfg.execpath
        if p.force or any([not os.path.exists(t[x]) for x in ['obam','abam','ubam']]): 
            nt = self.get_nthreads()
            extra = pc.get('extra','')
            idx = pc.index_prefix
            if paired:
                cmd = f"""{dart} -i {idx} -f {i['r1']} -f2 {i['r2']} -bo {t['obam']} -t {nt} {extra} \
                          | tr '\r' '\n' | tail -n 9 > {o['log']}"""
            else:
                cmd = f"""{dart} -i {idx} -f {i['r1']}               -bo {t['obam']} -t {nt} {extra} \
                          | tr '\r' '\n' | tail -n 9 > {o['log']}"""
            subprocesscall(cmd)
        # 2nd step: bam => aligned.bam, unmapped.fastq
        samtools = p['samtools.execpath']
        nt = self.get_nthreads('samtools')
        if p.force or any([not os.path.exists(o[x]) for x in ['sbam','sbai']]):
            inp = t['obam']
            if paired:
                cmd1 = f"{samtools} view -@ {nt} -f 2 {inp} -o {t['abam']}"
                cmd2 = f"{samtools} view -@ {nt} -F 2 {inp} -o {t['ubam']}"
            else:
                cmd1 = f"{samtools} view -@ {nt} -F 4 {inp} -o {t['abam']}"
                cmd2 = f"{samtools} view -@ {nt} -f 4 {inp} -o {t['ubam']}"
            subprocesscall(cmd1)
            subprocesscall(cmd2)
            os.unlink(inp)
            # convert unmapped.bam to fastq
            bedtools = p['bedtools.execpath']
            if paired:
                cmd = f"{bedtools} bamtofastq -i {t['ubam']} -fq {t['fq1']} -fq2 {t['fq2']}"
            else:
                cmd = f"{bedtools} bamtofastq -i {t['ubam']} -fq {t['fq']}"
            subprocesscall(cmd)
            tgts = [t['fq1'],t['fq2']] if paired else [t['fq']]
            self.compress(tgts)            
        # 3rd step: aligned.bam => aligned.sorted.bam
        if p.force or any([not os.path.exists(o[x]) for x in ['sbam']]):
            cmd = f"{samtools} sort {t['abam']} -o {o['sbam']} -@ {nt}"
            subprocesscall(cmd)
            os.unlink(t['abam'])
        # 4th step: bam index
        if p.force or (not os.path.exists(o['sbai'])):
            cmd = f"{samtools} index {o['sbam']}"
            subprocesscall(cmd)    

    def _run(self):
        p = self.p
        pc = self.pcfg
        i = self._inps
        t = self._tmps
        o = self._outs
        paired = self.paired
        dart = self.pcfg.execpath
        samtools = p.samtools.execpath
        bedtools = p.bedtools.execpath
        pigz = p.pigz.execpath
        nt = self.get_nthreads()
        nts = self.get_nthreads('samtools')
        ntp = self.get_nthreads('pigz')
        extra = pc.get('extra','')
        idx = pc.index_prefix
        if paired:
            cmd = f"""{dart} -i {idx} -f {i['r1']} -f2 {i['r2']} -bo {t['obam']} -t {nt} {extra} \
                          | tr '\r' '\n' | tail -n 9 > {o['log']}; \
                      {samtools} view -@ {nts} -f 2 {t['obam']} -o {t['abam']}; \
                      {samtools} view -@ {nts} -F 2 {t['obam']} -o {t['ubam']}; \
                      rm {t['obam']}; \
                      {bedtools} bamtofastq -i {t['ubam']} -fq {t['fq1']} -fq2 {t['fq2']}; \
                      {pigz} -p {ntp} --fast {t['fq1']}; \
                      {pigz} -p {ntp} --fast {t['fq2']}; \
                      {samtools} sort {t['abam']} -o {o['sbam']} -@ {nts}; \
                      rm {t['abam']}; \
                      {samtools} index {o['sbam']}; 
                   """
        else:
            cmd = f"""{dart} -i {idx} -f {i['r1']}               -bo {t['obam']} -t {nt} {extra} \
                          | tr '\r' '\n' | tail -n 9 > {o['log']}; \
                      {samtools} view -@ {nts} -F 4 {t['obam']} -o {t['abam']}; \
                      {samtools} view -@ {nts} -f 4 {t['obam']} -o {t['ubam']}; \
                      rm {t['obam']}; \
                      {bedtools} bamtofastq -i {t['ubam']} -fq {t['fq']}; \
                      {pigz} -p {ntp} --fast {t['fq1']}; \
                      {samtools} sort {t['abam']} -o {o['sbam']} -@ {nts}; \
                      rm {t['abam']}; \
                      {samtools} index {o['sbam']}; 
                   """
        subprocesscall(cmd)            

# dart: https://github.com/hsinnan75/DART
@task
def dart_scatter_index(cfg, indexname):
    "scatter dart indices, return modified config"
    if (cfg.nfs=='') or (cfg.ramdisk=='') or (cfg.nfs== cfg.ramdisk):
        return cfg
    idxpre = cfg[f'dart.{indexname}.index_prefix']
    nfshost = cfg.nfshost
    remotes = cfg.workers #[x for x in p0.workers if x!=nfshost]
    files = glob.glob(idxpre+'*')
    # copy index files to all workers (except nfs host)
    for host in remotes:
        srcdsts = [(x,x.replace(cfg.nfs,cfg.ramdisk)) for x in files]
        remote_put_files(host,srcdsts,cfg.sshuser)
    cfg1 = cfg.copy()
    idxpre1 = idxpre.replace(cfg.nfs,cfg.ramdisk)
    cfg1[f'dart.{indexname}.index_prefix'] = idxpre1    
    return cfg1
    



# Gene Expression Read Counts

## FeatureCounts

In [16]:
# export
class FeatureCounts(FileInFileOut):
    """
    Inputs:
        1. bam (p.bam)
    Outputs:
        1. counts (=>p.fccnts)
        2. readmap (=>p.fcmaps)
    """
    nickname = 'featurecounts'
    i_tmpls = dict(
        bam = '{bam}'
    )
    t_tmpls = dict(
        count = '{prefix}.featurecounts.txt',
        reads = '{prefix}.aligned.sorted.bam.featureCounts', # destination of each reads        
    )
    o_tmpls = dict(
        count = '{prefix}.featurecounts.txt.gz',
        reads = '{prefix}.aligned.sorted.bam.featureCounts.gz',
        log = '{prefix}.featurecounts.txt.summary',
    )
    
    def _read_stats(self):
        df = PD.read_table(self.outs['log'], comment='#', names=['field','value'], index_col=[0])
        return dict(df['value'])
    
    def _modify_param_specific(self):
        # swap fastq inputs
        pa = self.param
        o = self.outs
        pa.fccnts = o['count']
        pa.fcmaps = o['reads']
            
    def _run(self):
        pc = self.pcfg
        exe = pc.execpath
        i = self._inps
        t = self._tmps
        cmd = f"{exe} -R CORE -t exon -g gene_id --primary -a {pc['gtf']} -o {t['count']} {i['bam']}"
        subprocesscall(cmd)
        self.compress(t)
                        

# SmartSeq3

In [20]:
# export
class SS3CountTags(FileInFileOut):
    """
    Inputs:
        1. fastq (p.r1, p.r2) # assumes paired
    Outputs:
        1. csv (contents to p.info.ss3counttags)
    """
    nickname = 'ss3counttags'
    o_tmpls = dict(
        tab = '{prefix}-{history}-{tag}.txt'
    )
    def __init__(self, tagseq='ATTGCGCAATG', **kw):
        self.tagseq = tagseq
        self.revseq = revcomp(tagseq)
        super().__init__(**kw)
        
    def _read_stats(self):
        if hasattr(self, '_cntsdf'):
            return dict(self._cntsdf.sum(axis=0))
        df = PD.read_table(self.outs['txt'], index_col=[0])
        return dict(df.sum(axis=0))
        
    def _run(self):
        p = self.p
        fastq = FASTQP(p.r1,p.r2) # read all at once
        cnts = {k: {} for k in ['p1','p2','rp1','rp2']}
        for l1,l2 in fastq:
            s1,s2 = l1[1],l2[1]
            for k,s,t in [('p1',s1,tag),('p2',s2,tag),('rp1',s1,rtag),('rp2',s2,rtag)]:
                pos = s.find(t)
                cnts[k][pos] = cnts[k].get(pos,0) + 1 # count frequency of the position
        a = PD.DataFrame(cnts).fillna(0).sort_index()
        f = self._outs['txt']
        a.to_csv(f, header=True, index=True, sep='\t')
        os.chmod(f, 0o775)
        self._cntsdf = a



class SS3RemoveTags(FileInFileOut):
    """
    Inputs:
        1. fastq (p.r1, p.r2) # paried
    Outputs:
        1. fastq (=> p.r1, p.r2) # tag removed
        
    Remove TAG and associated adapter/UMI from 5' side of read1.
    If TAG is in r1 and rcTAG is in r2, then also remove 3' side from read2
    Swap original (long id) to simple id, also group by unique seq (uniq seq id)
    Record UMI, TAG position, uniq seq id, org id: as pandas dataframe.
        
    """
    nickname = 'ss3removetags'
    o_tmpls = dict(
        r1 = '{prefix}_1.fastq.gz',
        r2 = '{prefix}_2.fastq.gz',
        info = '{prefix}-info.txt.gz',
    )
    
    def __init__(self, tagseq='ATTGCGCAATG', **kw):
        self.tagseq = tagseq
        super().__init__(**kw)
        
    def _read_stats(self):
        dic = dict(
            infopath = self.outs['info'],
            stats = self._stats if hasattr(self, '_stats') else None,
        )
        return dic
    
    def _modify_param_specific(self):
        pa = self.param
        o = self.outs
        pa.r1 = o['r1']
        pa.r2 = o['r2']
        
    def _run(self):
        p = self.p
        pc = self.pcfg
        fastq = FASTQP(p.r1,p.r2)
        tag = self.tagseq
        rtag = revcomp(tag)
        lines1 = [] # filtered read 1 
        lines2 = [] # filtered read 2
        seqs = {} # seq => uid
        info = [] # rows (orgid, newid, uniqid, len, tagpos, umiseq)
        n = len(tag)
        nu = pc.get('umibp',8)
        minlen = pc.get('minlen',30)
        idx = -1
        for cid, (l1,l2) in enumerate(fastq):
            s1,s2 = l1[1][:-1],l2[1][:-1] # seq
            q1,q2 = l1[3][:-1],l2[3][:-1] # quality
            p1 = s1.find(tag)       
            p2 = s2.find(tag)
            umi = '' # default no umi
            if p1>=0: # NPA cleaved 5' end or using seqsite from FP
                st = p1 + n + nu
                umi = s1[st-nu:st] # umi seq
                s1 = s1[st:] # trim 5'
                q1 = q1[st:]
                rp2 = s2.find(rtag)
                if rp2>=0: # if the pair overlaps...
                    ed = rp2 - nu
                    s2 = s2[:ed] # trim 3'
                    q2 = q2[:ed]
            elif p2>=0: # NPB cleaved 5' end
                st = p2 + n + nu
                umi = s2[st-nu:st]
                s2 = s2[st:]
                q2 = q2[st:]
                rp1 = s1.find(rtag)
                if rp1>=0:
                    ed = rp1 - nu
                    s1 = s1[:ed]
                    q1 = q1[:ed]
            # new id = cid
            # uniq id? (s1,s2)=> newid of first occurence
            ovl,tot,sl1,sl2 = detect_overlap(s1,s2)
            if (len(s1)>=minlen) and (len(s2)>=minlen):
                uid = seqs.setdefault((s1,s2),cid)
                lines1.append(f'@{cid}\n{s1}\n+\n{q1}\n')
                lines2.append(f'@{cid}\n{s2}\n+\n{q2}\n')
                idx += 1 # count lines added
            else:
                uid = -1
            row = (l1[0],cid,uid,idx,tot,ovl,p1,p2,umi)
            info.append(row)
        # stats
        self._stats = {'org':len(info),'filtered':len(lines1)}
        # write
        # info
        infodf = PD.DataFrame(info, columns=['oid','nid','uid','idx','len','ovl','p1','p2','umi'])
        o = self._outs
        infodf.to_csv(o['info'], header=True, index=True, sep='\t', compression='gzip')
        os.chmod(o['info'], 0o775)
        # read1,read2
        gzip.open(o['r1'],'wt').write(''.join(lines1))
        gzip.open(o['r2'],'wt').write(''.join(lines2))
        os.chmod(o['r1'],0o775)
        os.chmod(o['r2'],0o775)
